In [9]:
import sys
import pathlib as pl

import gtfs_kit as gk

sys.path.append("../")

import gtfsr_kit as grk

%load_ext autoreload
%autoreload 2

DATA_DIR = pl.Path("../data")
%ls {DATA_DIR}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
test_gtfsr/  test_gtfs.zip  tripUpdates.pb  tripUpdates_short.json


In [4]:
# Read a feed

path = DATA_DIR/'tripUpdates.pb'
feed = grk.read_feed(path)

In [5]:
print(feed.header.timestamp)
print(grk.get_timestamp_str(feed))

print(feed.entity[0])
print(grk.feed_to_dict(feed)['entity'][0])

1417387583
2014-12-01T11:46:23
id: "2014-11-30:CXX:L019:7062"
trip_update {
  trip {
    trip_id: "8803145"
    start_date: "20141130"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 1
    arrival {
      delay: 0
      time: 1417386840
    }
    departure {
      delay: 19
      time: 1417386859
    }
    stop_id: "36992"
  }
  stop_time_update {
    stop_sequence: 3
    arrival {
      delay: -19
      time: 1417387001
    }
    departure {
      delay: -19
      time: 1417387001
    }
    stop_id: "46130"
  }
  stop_time_update {
    stop_sequence: 4
    arrival {
      delay: -12
      time: 1417387068
    }
    departure {
      delay: -12
      time: 1417387068
    }
    stop_id: "45551"
  }
  stop_time_update {
    stop_sequence: 5
    arrival {
      delay: -50
      time: 1417387090
    }
    departure {
      delay: -50
      time: 1417387090
    }
    stop_id: "41317"
  }
  stop_time_update {
    stop_sequence: 6
    arrival {
      delay: -3

In [6]:
# Extract delays

grk.extract_delays(feed)

,route_id,trip_id,stop_sequence,stop_id,arrival_delay,departure_delay
0,,10376364,1,39820,0.0,23.0
1,,10376364,2,158895,52.0,78.0
2,,10376364,3,158853,-24.0,-24.0
3,,10376364,4,41875,-34.0,-18.0
4,,10376364,5,41305,21.0,21.0
...,...,...,...,...,...,...
60069,,9297507,13,40186,177.0,177.0
60070,,9297507,14,38259,37.0,92.0
60071,,9297507,15,322154,44.0,44.0
60072,,9297507,16,38767,-16.0,-16.0


In [7]:
# Build augmented stop times

gtfsr_dir = DATA_DIR/'test_gtfsr'
gtfsr_feeds = [grk.read_feed(path, from_json=True) for path in gtfsr_dir.iterdir()]
gtfs_feed = gk.read_feed(DATA_DIR/'test_gtfs.zip', dist_units='km') # Good from 20160519
date = '20160519'

ast = grk.build_augmented_stop_times(gtfsr_feeds, gtfs_feed, date)

trip_id = '14005028723-20160512154122_v40.34'
ast[ast['trip_id'] == trip_id].T

,104,105,106,107,108,109,110,111,112,113,...,120,121,122,123,124,125,126,127,128,129
trip_id,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,...,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34
arrival_time,08:00:00,08:00:40,08:01:06,08:01:46,08:02:22,08:02:53,08:03:47,08:05:18,08:06:57,08:07:42,...,08:17:27,08:19:42,08:22:30,08:25:00,08:27:30,08:30:00,08:33:00,08:35:20,08:37:40,08:40:00
departure_time,08:00:00,08:00:40,08:01:06,08:01:46,08:02:22,08:02:53,08:03:47,08:05:18,08:07:00,08:07:42,...,08:17:27,08:20:00,08:22:30,08:25:00,08:27:30,08:30:00,08:33:00,08:35:20,08:37:40,08:40:00
stop_id,8000,8002,8004,8006,8008,8010,1049,1051,8491,8493,...,7115,7113,7111,7109,7107,7105,7099,7051,7049,7015
stop_sequence,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
stop_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pickup_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
drop_off_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_dist_traveled,0,0.225186,0.472539,0.765031,0.989229,1.17421,1.68721,2.34672,3.11742,3.45579,...,6.02873,6.3742,6.74371,6.98197,7.32772,7.63741,8.13787,8.541,8.67873,8.98601
arrival_delay,NaN,398,NaN,396,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Interpolate delays

f = grk.interpolate_delays(ast, dist_threshold=1)
f[f['trip_id'] == trip_id].T


,104,105,106,107,108,109,110,111,112,113,...,120,121,122,123,124,125,126,127,128,129
trip_id,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,...,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34,14005028723-20160512154122_v40.34
arrival_time,08:00:00,08:00:40,08:01:06,08:01:46,08:02:22,08:02:53,08:03:47,08:05:18,08:06:57,08:07:42,...,08:17:27,08:19:42,08:22:30,08:25:00,08:27:30,08:30:00,08:33:00,08:35:20,08:37:40,08:40:00
departure_time,08:00:00,08:00:40,08:01:06,08:01:46,08:02:22,08:02:53,08:03:47,08:05:18,08:07:00,08:07:42,...,08:17:27,08:20:00,08:22:30,08:25:00,08:27:30,08:30:00,08:33:00,08:35:20,08:37:40,08:40:00
stop_id,8000,8002,8004,8006,8008,8010,1049,1051,8491,8493,...,7115,7113,7111,7109,7107,7105,7099,7051,7049,7015
stop_sequence,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
stop_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pickup_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
drop_off_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_dist_traveled,0,0.225186,0.472539,0.765031,0.989229,1.17421,1.68721,2.34672,3.11742,3.45579,...,6.02873,6.3742,6.74371,6.98197,7.32772,7.63741,8.13787,8.541,8.67873,8.98601
arrival_delay,398,398,397,396,385,376,352,320,283,266,...,142,126,108,97,80,65,41,21,15,0
